# Sandbox

Space for Python code development...

***

In [ ]:
import numpy as np
import xarray as xr
import yaml
import sys
from matplotlib import pyplot as plt
from tqdm import tqdm

# Import modules from path
sys.path.append('../modules/')
import calc_transformation_budget as trtools
import watermasstools as wmttools

plt.rcParams.update({'font.size': 11, 'axes.titlesize': 'medium'})

***

### Code development starts here ...